In [2]:
#export
import torch
from torch import tensor
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [4]:
#export
import pickle,gzip,math,torch,matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
from fastai import datasets

In [6]:
from pathlib import Path
from torch.nn import init

In [8]:
MNIST_URL = "http://deeplearning.net/data/mnist/mnist.pkl"
path = datasets.download_data(MNIST_URL, ext='.gz')
path

PosixPath('/Users/abhinavverma/.fastai/data/mnist.pkl.gz')

In [9]:
#export
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))
def normalize(x,m,s):
    return (x-m)/s

In [10]:
x_train,y_train,x_valid,y_valid = get_data()

In [12]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [13]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [14]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [15]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [16]:
test_near_zero(train_mean)

In [17]:
test_near_zero(1-x_train.std())

In [18]:
??nn.Linear

We will use kaiming_uniform to initialize the weights as the mnist is a small example. so this is going with the defaults

In [159]:
#Global variables
n,m = x_train.shape
c = y_train.max().item()+1
nh = 50
n,m,c

(50000, 784, 10)

In [20]:
#model class is a callable class so it can be called like a function
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [26]:
#Model class
model = Model(m,nh,10)

In [24]:
model(x_train)

torch.Size([50000, 10])

In [27]:
#Global variables
pred = model(x_train)

##### We have already seen the importance of the softmax function. The softmax function is used to calculate the probabailities of a category in a multi-class class classification. 

In [25]:
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

In [28]:
softmax_pred = log_softmax(pred)

In [29]:
softmax_pred

tensor([[-2.8977, -2.2487, -2.8949,  ..., -2.2749, -1.8786, -2.6079],
        [-2.7164, -2.1471, -2.6864,  ..., -2.2885, -1.9710, -2.7302],
        [-2.5435, -2.4374, -2.8221,  ..., -2.5087, -2.1419, -2.0730],
        ...,
        [-2.3768, -2.5188, -2.7223,  ..., -2.3439, -2.0737, -2.3840],
        [-2.6748, -2.6027, -2.4636,  ..., -2.2222, -2.0227, -2.3758],
        [-2.5432, -2.6758, -2.2384,  ..., -2.2847, -2.0998, -2.5639]],
       grad_fn=<LogBackward>)

Now for cross entropy

In [30]:
def nll(pred,label):
    return -pred[range(label.shape[0]),label].mean()

In [32]:
nll(softmax_pred,y_train)

tensor(2.3521, grad_fn=<NegBackward>)

In [39]:
??torch.max

In [40]:
#this is calculated based on the log sum exp trick, a is the max value
def logsumexp(x):
    a = x.max(-1)[0]
    return a + (x-a[:,None]).exp().sum(-1).log()

In [41]:
#our log softmax
def log_softmax(x): return x - x.logsumexp(-1,keepdim=True)

In [42]:
softmax_pred_exp = log_softmax(pred)

In [44]:
#test_near(nll(log_softmax(pred), y_train), loss)

In PyTorch, F.log_softmax and F.nll_loss are combined in one optimized function, F.cross_entropy.

In [45]:
test_near_zero(F.cross_entropy(pred, y_train)-nll(log_softmax(pred), y_train))

So the implementation is pretty accurate

In [117]:
#Global variable
loss_func = F.cross_entropy

In [46]:
#export
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

We will use nn.Sequential to add layers to a Pytorch module.

In [47]:
#new Model
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [48]:
#export
from torch import optim

In [55]:
#Access lr or other parameters from optimizer.useful for modifying during learning
optim.SGD(model.parameters(), lr=0.5).param_groups[0]['lr']

0.5

In [60]:
#returns model and optimizer
def get_model():
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))
    return model, optim.SGD(model.parameters(), lr=0.5)

Since it is clunky to iterate through mini batches and specifying batch size all the time we will specify our dataset class and learn more about Pytorch Dataset,Dataloaders and RandomSamplers

In [124]:
#export
class Dataset():
    def __init__(self, x, y): self.x,self.y = x,y
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i],self.y[i]

DataLoaders are added to make the training loop much cleaner

In [57]:
class DataLoader():
    def __init__(self, ds, bs): self.ds,self.bs = ds,bs
    def __iter__(self):
        for i in range(0, len(self.ds), self.bs): yield self.ds[i:i+self.bs]

In [97]:
model, _ = get_model()

In [87]:
next(model.named_children())

('0', Linear(in_features=784, out_features=50, bias=True))

In [96]:
#How to access the weights of every layer. Obviously there has to be an optimized way
#next(model[0].parameters())[0].data

In [100]:
#Change weights using apply method of Pytorch
def init_weights(m):
    print(m)
    if type(m) == nn.Linear:
        
        m.weight.data.fill_(1.0)
        print(m.weight)
    
        
        
#model.apply(init_weights)

#### Random sampling
We want our training set to be in a random order, and that order should differ each iteration. But the validation set shouldn't be randomized.

In [101]:
class Sampler():
    def __init__(self, ds, bs, shuffle=False):
        self.n,self.bs,self.shuffle = len(ds),bs,shuffle
        
    def __iter__(self):
        self.idxs = torch.randperm(self.n) if self.shuffle else torch.arange(self.n)
        for i in range(0, self.n, self.bs): yield self.idxs[i:i+self.bs]

In [125]:
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
assert len(train_ds)==len(x_train)
assert len(valid_ds)==len(x_valid)

In [103]:
small_ds = Dataset(*train_ds[:10])

In [104]:
small_ds

In [105]:
s = Sampler(small_ds,3,False)
[o for o in s]

[tensor([0, 1, 2]), tensor([3, 4, 5]), tensor([6, 7, 8]), tensor([9])]

In [106]:
s = Sampler(small_ds,3,True)
[o for o in s]

[tensor([5, 6, 1]), tensor([3, 4, 7]), tensor([9, 2, 0]), tensor([8])]

In [107]:
#Dataloader with a custom Random sampleer
def collate(b):
    xs,ys = zip(*b)
    return torch.stack(xs),torch.stack(ys)

class DataLoader():
    def __init__(self, ds, sampler, collate_fn=collate):
        self.ds,self.sampler,self.collate_fn = ds,sampler,collate_fn
        
    def __iter__(self):
        for s in self.sampler: yield self.collate_fn([self.ds[i] for i in s])

But we can also use Pytorch's built in method of data sampling. It's the same as normal dataloader and sampler we defined above but it has a num_workers which allows for parallel processing which will be necessary in the future hence we use them instead

In [126]:
#export
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [127]:
#Global variable
bs=64

In [128]:
train_dl = DataLoader(train_ds, bs, sampler=RandomSampler(train_ds), collate_fn=collate)
valid_dl = DataLoader(valid_ds, bs, sampler=SequentialSampler(valid_ds), collate_fn=collate)

In [129]:
#fit before Callback
def fit():
    for epoch in range(epochs):
        for xb,yb in train_dl:
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

In [130]:
#Validation fit before callback
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm / dropout
        model.train()
#         print(model.training)
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
#         print(model.training)
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [131]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [132]:
#get Dataloader and define model
train_dl,valid_dl = get_dls(train_ds, valid_ds, bs)
model,opt = get_model()


In [133]:
#fit
loss,acc = fit(5, model, loss_func, opt, train_dl, valid_dl)

0 tensor(0.2402) tensor(0.9323)
1 tensor(1.2160) tensor(0.7853)
2 tensor(0.2268) tensor(0.9414)
3 tensor(0.1805) tensor(0.9487)
4 tensor(0.1883) tensor(0.9498)


In [134]:
assert acc > 0.9

The train_dl and valid_dl can be combined together as it's better to combine things into a class than call it from a function. So the train_dl and valid_dl are combined into a databunch

In [135]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c
        
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [141]:
a = zip(*get_dls(train_ds, valid_ds, bs))

In [148]:
get_dls(train_ds, valid_ds, bs)

(<torch.utils.data.dataloader.DataLoader at 0x1a246e13c8>,
 <torch.utils.data.dataloader.DataLoader at 0x1a25539c18>)

In [162]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c) #c = y_train.max()+1

Let's continue our class factorization by combining get_model function and loss_func and Databunch into a Learner class.

In [163]:
#export
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

In [165]:
data.c

10

In [164]:
learn = Learner(*get_model(data), loss_func, data)

In [166]:
#new fit
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += learn.loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(learn.data.valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [167]:
loss,acc = fit(4, learn)

0 tensor(0.4959) tensor(0.8421)
1 tensor(0.2965) tensor(0.9240)
2 tensor(0.2456) tensor(0.9360)
3 tensor(0.2628) tensor(0.9349)


In [168]:
"""
loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()
"""
#This is one batch of training so we can combine it in a function one_batch and all callbacks at every step

'\nloss = learn.loss_func(learn.model(xb), yb)\n            loss.backward()\n            learn.opt.step()\n            learn.opt.zero_grad()\n'

In [169]:
def one_batch(xb,yb,cb):
    """
    loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()
    The learner object is passed to the callback class. So it's accessed from the callback class
    We add callbacks before each and every step of the fit function
    cb.begin_batch - Called on batch begin
    cb.learn.loss_func The learn object is called from the loss function
    cb.learn.model(xb) - forward pass
    cb.after_backward() - after the backward pass
    cb.learn.opt.step() -optimizer called from the learner obj accessed from the callback object
    cb.after_step() after opt.zero_grad()
    """
    if not cb.begin_batch(xb,yb): return
    loss = cb.learn.loss_func(cb.learn.model(xb), yb)
    if not cb.after_loss(loss): return
    loss.backward()
    if cb.after_backward(): cb.learn.opt.step()
    if cb.after_step(): cb.learn.opt.zero_grad()

In [170]:
def all_batches(dl,cb):
    """
    This function replaces the second for loop in the original fit function
    for xb,yb in learn.data.train_dl:
    cb.do_stop callback function for handling after the training loop is complete
    """
    for xb,yb in dl:
        one_batch(xb, yb, cb)
        if cb.do_stop(): return

In [172]:
def fit(epochs, learn, cb):
    """
    cb.begin_fit -  on begin of fit function
    cb.begin_epoch -  on start of epoch
    cb.begin_validate - on validtion step begin
    cb.do_stop - called after epoch ends to stop training
    """
    if not cb.begin_fit(learn): return
    for epoch in range(epochs):
        if not cb.begin_epoch(epoch): continue
        all_batches(learn.data.train_dl, cb)
        
        if cb.begin_validate():
            with torch.no_grad(): all_batches(learn.data.valid_dl, cb)
        if cb.do_stop() or not cb.after_epoch(): break
    cb.after_fit()

Callback is an abstract class. It acts like a class which has factory methods which are then implemented by the various callback classes. 

In [173]:
class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

Callback handler class is used to take a list of the callbacks and implement them in the fit function. So many different callbacks can be implemented. Later we will add an order to it to initialize the order of the callback classes being executed

In [176]:
class CallbackHandler():
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []
    
    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
        for cb in self.cbs: res = res and cb.begin_fit(learn)
        return res

    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        return res

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res
    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res = res and cb.after_step()
        return res
    
    def do_stop(self):
        try:     return self.learn.stop
        finally: self.learn.stop = False

In [177]:
class TestCallback(Callback):
    
    def begin_epoch(self,epoch):
        super().begin_epoch(epoch)
        print("Beginning epoch")
        print(self.learn.data.c)
        print(type(self.learn))
        return True
        
        
    
    def begin_fit(self,learn):
        super().begin_fit(learn)
        self.n_iters = 0
        print("iteration step begins")
        return True
        
    def after_step(self):
        self.n_iters += 1
        print(self.n_iters)
        print("Steps over")
        if self.n_iters>=10: self.learn.stop = True
        return True

In [178]:
fit(1, Learner(*get_model(data), loss_func, data), cb=CallbackHandler([TestCallback()]))

iteration step begins
Beginning epoch
10
<class '__main__.Learner'>
1
Steps over
2
Steps over
3
Steps over
4
Steps over
5
Steps over
6
Steps over
7
Steps over
8
Steps over
9
Steps over
10
Steps over


Let's see if we can make things simpler and more flexible, so that a single class has access to everything and can change anything at any time. The fact that we're passing cb to so many functions is a strong hint they should all be in the same class!